# NORTHWIND DATABASE

El **objetivo** de este ejercicio es generar una serie de consultas SQL que permitan extraer información relevante de la base de datos y luego exportar los resultados para construir dashboards o visualizaciones que proporcionen insights valiosos para la toma de decisiones empresariales.

La base de datos Northwind es una base de datos de ejemplo clásica que simula las operaciones de una empresa minorista. Contiene tablas relacionales que representan diferentes aspectos del negocio, como productos, clientes, pedidos y empleados. Utilizando consultas SQL, podrás explorar y analizar estos datos para comprender mejor el desempeño del negocio en diferentes áreas.

Una vez que hayas realizado las consultas SQL y obtenido los resultados, podrás utilizarlos para herramientas de visualización de datos como Tableau, Power BI, Streamlit o cualquier otra herramienta de tu elección. 

In [9]:
![imagen](./img/northwind-er-diagram.png) 

"[imagen]" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


Tendrás que elegir uno de los retos a resolver:

In [10]:
pip install ibis-framework[sqlite]

Note: you may need to restart the kernel to use updated packages.


In [11]:
from sqlalchemy import create_engine 

engine = create_engine('sqlite:///C:/Users/rodri/OneDrive/Escritorio/DATA_SCIENCE/CODIGO_CLASES/TheBridge_DSPT-1/2-Data_Analysis/6-Bases_de_datos/SQL/Práctica/Northwind/data/northwind.db', echo=False)
connection = engine.connect() 

In [12]:
# 1.- CON 'SELECT' SELECCIONO TODOS LOS CAMPOS QUE VOY A NECESITAR 
# 2.- LUEGO, CON 'FROM' COMIENZO LA ORDER DE JUNTAR LAS TABLAS QUE CONTIENEN LOS CAMPOS QUE NECESITO Y LUEGO 'INNER JOIN' 

In [13]:
import pandas as pd 

consulta = """
SELECT datetime(o.OrderDate) as OrderDate, o.ShipRegion, od.Quantity*od.UnitPrice*(1-od.Discount) AS Total, p.ProductName 
FROM Orders o
INNER JOIN [Order Details] od ON od.OrderID == o.OrderID
INNER JOIN Products p ON p.ProductID = od.ProductID
""" 

data_df = pd.read_sql(consulta, con=connection)
data_df 

,OrderDate,ShipRegion,Total,ProductName
0,2016-07-04 00:00:00,Western Europe,168.00,Queso Cabrales
1,2016-07-04 00:00:00,Western Europe,98.00,Singaporean Hokkien Fried Mee
2,2016-07-04 00:00:00,Western Europe,174.00,Mozzarella di Giovanni
3,2016-07-05 00:00:00,Western Europe,167.40,Tofu
4,2016-07-05 00:00:00,Western Europe,1696.00,Manjimup Dried Apples
...,...,...,...,...
609278,2023-01-10 05:40:51,British Isles,806.00,Ikura
609279,2023-01-10 05:40:51,British Isles,216.00,Spegesild
609280,2023-01-10 05:40:51,British Isles,93.69,Gumbär Gummibärchen
609281,2023-01-10 05:40:51,British Isles,1053.60,Schoggi Schokolade


In [14]:
data_df.dtypes 

OrderDate       object
ShipRegion      object
Total          float64
ProductName     object
dtype: object

In [15]:
data_df["OrderDate"] = pd.to_datetime(data_df['OrderDate']) # CON 'Pandas' CON 'to_datetime' CONVIERTE EL CAMPO 'OrderDate' A FORMATO 'datetime' 
data_df.set_index("OrderDate", inplace=True) # PONE EL CAMPO 'OrderDate' COMO INDICE PORQUE LE PIDEN LOS PRODUCTOS POR FECHAS  

In [16]:
data_df.index 

DatetimeIndex(['2016-07-04 00:00:00', '2016-07-04 00:00:00',
               '2016-07-04 00:00:00', '2016-07-05 00:00:00',
               '2016-07-05 00:00:00', '2016-07-08 00:00:00',
               '2016-07-08 00:00:00', '2016-07-08 00:00:00',
               '2016-07-08 00:00:00', '2016-07-08 00:00:00',
               ...
               '2023-01-10 05:40:51', '2023-01-10 05:40:51',
               '2023-01-10 05:40:51', '2023-01-10 05:40:51',
               '2023-01-10 05:40:51', '2023-01-10 05:40:51',
               '2023-01-10 05:40:51', '2023-01-10 05:40:51',
               '2023-01-10 05:40:51', '2023-01-10 05:40:51'],
              dtype='datetime64[ns]', name='OrderDate', length=609283, freq=None)

In [17]:
data_df.head() # AQUI SE PUEDE VER QUE EL CAMPO 'OrderDate' ESTA CONVERTIDO EN EL INDICE DE LA TABLA DE DATOS 

,ShipRegion,Total,ProductName
OrderDate,,,
2016-07-04,Western Europe,168.0,Queso Cabrales
2016-07-04,Western Europe,98.0,Singaporean Hokkien Fried Mee
2016-07-04,Western Europe,174.0,Mozzarella di Giovanni
2016-07-05,Western Europe,167.4,Tofu
2016-07-05,Western Europe,1696.0,Manjimup Dried Apples


In [18]:
data_df["ShipRegion"].unique() 

array(['Western Europe', 'South America', 'Central America',
       'North America', 'Northern Europe', 'Scandinavia',
       'Southern Europe', 'British Isles', 'Eastern Europe'], dtype=object)

In [19]:
reg_prod_total = data_df.groupby(["ShipRegion"]).resample("BY").sum("Total")
reg_prod_total # AGRUPA POR "ShipRegion", Y LA SUMA DEL CAMPO "Total" (DE LA TABLA ANTERIOR) LA HACE CONSIDERANDO TODOS LOS DATOS DEL AÑO 'resample("BY")'  

Total
ShipRegion     OrderDate              
British Isles  2012-12-31   1638348.80
               2013-12-31   3187710.17
               2014-12-31   3872600.66
               2015-12-31   3479483.72
               2016-12-30   3555466.03
...                                ...
Western Europe 2019-12-31  11560187.87
               2020-12-31  12029549.51
               2021-12-31  11181004.58
               2022-12-30  11075892.76
               2023-12-29   9967003.48

[108 rows x 1 columns]

In [20]:
reg_prod_total.reset_index(inplace=True) 
reg_prod_total # QUITA EL INDICE QUE HABIA FIJADO PARA SACAR LA TABLA ANTERIOR 

,ShipRegion,OrderDate,Total
0,British Isles,2012-12-31,1638348.80
1,British Isles,2013-12-31,3187710.17
2,British Isles,2014-12-31,3872600.66
3,British Isles,2015-12-31,3479483.72
4,British Isles,2016-12-30,3555466.03
...,...,...,...
103,Western Europe,2019-12-31,11560187.87
104,Western Europe,2020-12-31,12029549.51
105,Western Europe,2021-12-31,11181004.58
106,Western Europe,2022-12-30,11075892.76


In [21]:
reg_prod_total["Year"] = reg_prod_total["OrderDate"].dt.year 
reg_prod_total # PARA SACAR EL AÑO Y COMO EL DATO ES DEL TIPO 'date' UTILIZA EL COMANDO 'dt' 
# ASÍ SE TRABAJA CON AÑOS COMPLETOS EN LUGAR DE FECHAS, QUE HAY MUCHAS DENTRO DE UN AÑO, SE CREA EL CAMPO 'Year' 
# '.str' SI ES CADENA DE TEXTO, 

,ShipRegion,OrderDate,Total,Year
0,British Isles,2012-12-31,1638348.80,2012
1,British Isles,2013-12-31,3187710.17,2013
2,British Isles,2014-12-31,3872600.66,2014
3,British Isles,2015-12-31,3479483.72,2015
4,British Isles,2016-12-30,3555466.03,2016
...,...,...,...,...
103,Western Europe,2019-12-31,11560187.87,2019
104,Western Europe,2020-12-31,12029549.51,2020
105,Western Europe,2021-12-31,11181004.58,2021
106,Western Europe,2022-12-30,11075892.76,2022


In [22]:
import plotly.express as px 

fig = px.line(reg_prod_total, x="Year", y='Total', color="ShipRegion")
fig.show() 

In [23]:
# AGRUPA LOS DATOS POR: AÑO, MES, REGION Y PRODUCTO; 
# SE LE SACA EL AÑO Y EL MES AL CAMPO 'OrderDate', ESO LA BASE DE DATOS LO SABE HACER
# OBTIENE LOS CAMPOS 'Region' Y 'Product'
# SUMA LOS PRECIOS UNITARIOS, LOS ORDENA DE FORMA DESCENDENTE    

In [24]:
consulta = '''
SELECT strftime('%Y', OrderDate) AS Year,
       strftime('%m', OrderDate) AS Month,
       ShipRegion AS Region,
       ProductName AS Product,
       SUM(Precio_Total) AS Total 
FROM (
        SELECT datetime(o.OrderDate) as OrderDate, 
        o.ShipRegion, p.ProductName, 
        od.Quantity*od.UnitPrice*(1-od.Discount) AS Precio_total
        FROM Orders o
        INNER JOIN [Order Details] od ON od.OrderID = o.OrderID
        INNER JOIN Products p ON p.ProductID = od.ProductID
)
GROUP BY Year, Month, Region, Product
ORDER BY Total DESC 
'''

data_df = pd.read_sql(consulta, con=connection) 
data_df 

,Year,Month,Region,Product,Total
0,2014,12,Western Europe,Côte de Blaye,222130.5
1,2015,08,Western Europe,Côte de Blaye,220022.5
2,2016,12,Western Europe,Côte de Blaye,209746.0
3,2013,07,Western Europe,Côte de Blaye,209482.5
4,2018,08,Western Europe,Côte de Blaye,192882.0
...,...,...,...,...,...
86605,2018,09,Scandinavia,Guaraná Fantástica,4.5
86606,2018,10,Northern Europe,Guaraná Fantástica,4.5
86607,2021,05,Eastern Europe,Guaraná Fantástica,4.5
86608,2023,08,Scandinavia,Guaraná Fantástica,4.5


In [25]:
product_profit = data_df.groupby(["Product", "Region"]).agg(meanTotal=("Total", "mean"), stdTotal=("Total", "std")).sort_values 
product_profit 

# HACE UNA AGRUPACION POR 'Product' Y 'Region'
# CALCULA LA MEDIA DEL PRECIO TOTAL Y LA DESVIACION ESTANDAR DEL PRECIO TOTAL 

<bound method DataFrame.sort_values of                                   meanTotal     stdTotal
Product       Region                                    
Alice Mutton  British Isles     5179.287313  2787.205289
              Central America   3407.300000  1868.169309
              Eastern Europe    1425.478261   849.683568
              North America    10596.377426  3591.610332
              Northern Europe   3048.251908  1684.280592
...                                     ...          ...
Zaanse koeken Northern Europe    701.004264   441.279471
              Scandinavia        541.913043   392.283532
              South America     2527.635662   897.178087
              Southern Europe   1351.514706   577.580365
              Western Europe    4252.961397  1253.503667

[693 rows x 2 columns]>

In [69]:
# SE HIZO UN FILTRO DE LOS DATOS DE LA TABLA ANTERIOR, ESTA TABLA INCLUYE SOLAMENTE LOS DATOS DE "Western Europe" 

region = "Western Europe" # Europa occidental 

region_profit = product_profit[product_profit["Region"] == region]
region_profit

TypeError: 'method' object is not subscriptable

In [ ]:
fig = px.bar(region_profit, x="Product", y="meanTotal", error_y="stdTotal")
fig.show() 

### **1. Análisis de Ventas por Región**
Analiza las ventas de productos por región. El objetivo es identificar las regiones que generan más ingresos y las tendencias de ventas a lo largo del tiempo en esas regiones. 
Pista: Utiliza gráficos de barras y líneas para mostrar la evolución de las ventas, así como mapas geográficos para visualizar la distribución de ingresos por región. 

### **2. Análisis de Desempeño de Productos**
Analiza el desempeño de los productos de la empresa. Examina las ventas de productos por categoría, identificando los productos más vendidos y los menos vendidos. 

Pista: Utiliza gráficos de barras y de tarta para visualizar la participación de mercado de cada categoría y los ingresos generados por los productos individuales. 

In [ ]:
pip install ibis-framework[sqlite]

Note: you may need to restart the kernel to use updated packages.


In [27]:
import ibis 

# Para ejecución interactiva 
ibis.options.interactive = True # PERMITE UNA INTERACCION DIRECTA CON LA BASE DE DATOS 

con = ibis.sqlite.connect('C:/Users/rodri/OneDrive/Escritorio/DATA_SCIENCE/CODIGO_CLASES/TheBridge_DSPT-1/2-Data_Analysis/6-Bases_de_datos/SQL/Práctica/Northwind/data/northwind.db') 

In [28]:
# LAS TABLAS 'categories' Y 'products'
# JUNTA AMBAS TABLAS EN EL CAMPO EN COMÚN 'CategoryID' 

categories = con.table("Categories")
products = con.table("Products") 

joined_products = categories.join(products, "CategoryID")
joined_products 

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ CategoryID ┃ CategoryName ┃ Description                                                ┃ Picture                                                                                                                                                                                                                                                               ┃ ProductID ┃ ProductName                     ┃ SupplierID ┃ QuantityPerUnit     ┃ UnitPrice ┃ UnitsInStock ┃ UnitsOnOrder ┃ ReorderLevel ┃ Discontinued ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ int64      │ string       │ string                                                     │ binary                                                                                                                                                                                                                                                                │ int64     │ string                          │ int64      │ string              │ decimal   │ int64        │ int64        │ int64        │ string       │
├────────────┼──────────────┼────────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────┼─────────────────────────────────┼────────────┼─────────────────────┼───────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│          1 │ Beverages    │ Soft drinks, coffees, teas, beers, and ales                │ b"\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00d\x00d\x00\x00\xff\xec\x00\x11Ducky\x00\x01\x00\x04\x00\x00\x00P\x00\x00\xff\xee\x00&Adobe\x00d\xc0\x00\x00\x00\x01\x03\x00\x15\x04\x03\x06\n\r\x00\x00\nc\x00\x00\x10\xe8\x00\x00\x19\xf4\x00\x00'\xa4\xff"+10071  │         1 │ Chai                            │          1 │ 10 boxes x 20 bags  │     18.00 │           39 │            0 │           10 │ 0            │
│          1 │ Beverages    │ Soft drinks, coffees, teas, beers, and ales                │ b"\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00d\x00d\x00\x00\xff\xec\x00\x11Ducky\x00\x01\x00\x04\x00\x00\x00P\x00\x00\xff\xee\x00&Adobe\x00d\xc0\x00\x00\x00\x01\x03\x00\x15\x04\x03\x06\n\r\x00\x00\nc\x00\x00\x10\xe8\x00\x00\x19\xf4\x00\x00'\xa4\xff"+10071  │         2 │ Chang                           │          1 │ 24 - 12 oz bottles  │     19.00 │           17 │           40 │           25 │ 0            │
│          2 │ Condiments   │ Sweet and savory sauces, relishes, spreads, and seasonings │ b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00d\x00d\x00\x00\xff\xec\x00\x11Ducky\x00\x01\x00\x04\x00\x00\x00L\x00\x00\xff\xee\x00&Adobe\x00d\xc0\x00\x00\x00\x01\x03\x00\x15\x04\x03\x06\n\r\x00\x00\n\xda\x00\x00\x14\xef\x00\x00 ,\x00\x00/H\xff'+12027        │         3 │ Aniseed Syrup                   │          1 │ 12 - 550 ml bottles │     10.0

In [29]:
# DEVUELVE EL TIPO DE CADA CAMPO 

products.schema()

ibis.Schema {
  ProductID        !int64
  ProductName      !string
  SupplierID       int64
  CategoryID       int64
  QuantityPerUnit  string
  UnitPrice        decimal
  UnitsInStock     int64
  UnitsOnOrder     int64
  ReorderLevel     int64
  Discontinued     !string
}

In [30]:
# DEVUELVE LOS CINCO PRIMEROS DATOS DE LOS CAMPOS 'ProductID' Y 'ProductName' 

products[["ProductID","ProductName"]].head(5) 

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ProductID ┃ ProductName                  ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ !int64    │ !string                      │
├───────────┼──────────────────────────────┤
│         1 │ Chai                         │
│         2 │ Chang                        │
│         3 │ Aniseed Syrup                │
│         4 │ Chef Anton's Cajun Seasoning │
│         5 │ Chef Anton's Gumbo Mix       │
└───────────┴──────────────────────────────┘

In [65]:
ibis.options.interactive = False 

products[["ProductID", "ProductName"]].head(5) # MUESTRA LA SQL QUE ESTÁ LANZANDO LA BASE DE DATOS, YA NO MUESTRA LOS DATOS PORQUE ELIMINÉ LA INTERACCIÓN 'ibis.options.interactive = 'false''

r0 := DatabaseTable: Products
  ProductID       !int64
  ProductName     !string
  SupplierID      int64
  CategoryID      int64
  QuantityPerUnit string
  UnitPrice       decimal
  UnitsInStock    int64
  UnitsOnOrder    int64
  ReorderLevel    int64
  Discontinued    !string

r1 := Project[r0]
  ProductID:   r0.ProductID
  ProductName: r0.ProductName

Limit[r1, n=5]

In [32]:
# CUANDO NO SE ESTÁ INTERACTUANDO CON LA BASE DE DATOS, CUANDO YO INGRESO INSTRUCCIONES DE COMANDOS EN PANDAS, IBIS LO DEVUELVE O CONVIERTE A CONSULTAS DEL TIPO 'SQL' 

ibis.to_sql(products[["ProductID", "ProductName"]].head(5)) 

```sql
SELECT
  "t0"."ProductID",
  "t0"."ProductName"
FROM "Products" AS "t0"
LIMIT 5
```

Pasamos a Pandas (en nuestra máquina) 

In [39]:
category_df = joined_products.select(["CategoryName", "ProductName"]).to_pandas() # 'to_pandas() TRAE LA CONSULTA A UN DATAFRAME DE PANDAS 
category_df.rename(columns={"ProductName": "Product"}, inplace=True) # EL CAMPO 'ProductName' LO RENOMBRA COMO 'Product' 
category_df.set_index("Product", inplace=True) # ESTABLECE EL INDICE COMO EL CAMPO 'product' 

In [66]:
# ESTO YA SE EJECUTA EN EL ORDENADOR, NO HAY INTERACCION CON LA BASE DE DATOS 

product_profit.reset_index(inplace=True) 
product_profit.set_index("Product", inplace=True) 

AttributeError: 'function' object has no attribute 'reset_index'

In [41]:
# ES UN DATAFRAME DE PANDAS 

product_profit 

<bound method DataFrame.sort_values of                                   meanTotal     stdTotal
Product       Region                                    
Alice Mutton  British Isles     5179.287313  2787.205289
              Central America   3407.300000  1868.169309
              Eastern Europe    1425.478261   849.683568
              North America    10596.377426  3591.610332
              Northern Europe   3048.251908  1684.280592
...                                     ...          ...
Zaanse koeken Northern Europe    701.004264   441.279471
              Scandinavia        541.913043   392.283532
              South America     2527.635662   897.178087
              Southern Europe   1351.514706   577.580365
              Western Europe    4252.961397  1253.503667

[693 rows x 2 columns]>

In [67]:
category_df.join(product_profit).sort_values("meanTotal", ascending=False).head(10) 

TypeError: 'method' object is not iterable

In [43]:
type(category_df) 

pandas.core.frame.DataFrame

In [44]:
type(joined_products) 

ibis.expr.types.joins.Join

In [45]:
# LO CONVIERTE A CONSULTAS DEL TIPO SQL 

ibis.to_sql(joined_products) 

```sql
SELECT
  "t2"."CategoryID",
  "t2"."CategoryName",
  "t2"."Description",
  "t2"."Picture",
  "t3"."ProductID",
  "t3"."ProductName",
  "t3"."SupplierID",
  "t3"."QuantityPerUnit",
  "t3"."UnitPrice",
  "t3"."UnitsInStock",
  "t3"."UnitsOnOrder",
  "t3"."ReorderLevel",
  "t3"."Discontinued"
FROM "Categories" AS "t2"
INNER JOIN "Products" AS "t3"
  ON "t2"."CategoryID" = "t3"."CategoryID"
```

In [68]:
con.table("Orders") 

KeyError: <Type.DATETIME: 'DATETIME'>

En este caso estamos obligados a realizar una 'select' explícita para extraer los datos como si fuera texto y procesarlo mediante 'parsers locales'. 

In [47]:
orders = con.sql("SELECT CAST(OrderDate as Text), OrderID FROM Orders")
orders # YA NO TIENE FORMATO 'datetime' QUE ESTABA DANDO ERROR, AHORA TIENE FORMATO 'string' 

SQLQueryResult
  query:
    SELECT CAST(OrderDate as Text), OrderID FROM Orders
  schema:
    CAST(OrderDate as Text) string
    OrderID                 int64

In [ ]:
# Si no esta en modo interactivo, será necesario el 'to_pandas()' 

In [48]:
orders.head(4) # DEVUELVE UNA CONSULTA DEL TIPO SQL 

r0 := SQLQueryResult
  query:
    SELECT CAST(OrderDate as Text), OrderID FROM Orders
  schema:
    CAST(OrderDate as Text) string
    OrderID                 int64

Limit[r0, n=4]

In [49]:
orders.head(4).to_pandas()  

,CAST(OrderDate as Text),OrderID
0,2016-07-04,10248
1,2016-07-05,10249
2,2016-07-08,10250
3,2016-07-08,10251


In [50]:
order_details = con.table("Order Details") 
order_details # DEVUELVE LOS TIPOS DE DATOS QUE CONTIENE LA TABLA 

DatabaseTable: Order Details
  OrderID   !int64
  ProductID !int64
  UnitPrice !decimal
  Quantity  !int64
  Discount  !float64

In [51]:
p_o_details = orders.join(order_details, ["OrderID"], how="inner").join(products, ["ProductID"]).join(categories, ["CategoryID"])
ibis.to_sql(p_o_details)  

```sql
SELECT
  "t4"."CAST(OrderDate as Text)",
  "t4"."OrderID",
  "t5"."ProductID",
  "t5"."UnitPrice",
  "t5"."Quantity",
  "t5"."Discount",
  "t6"."ProductName",
  "t6"."SupplierID",
  "t6"."CategoryID",
  "t6"."QuantityPerUnit",
  "t6"."UnitPrice" AS "UnitPrice_right",
  "t6"."UnitsInStock",
  "t6"."UnitsOnOrder",
  "t6"."ReorderLevel",
  "t6"."Discontinued",
  "t7"."CategoryName",
  "t7"."Description",
  "t7"."Picture"
FROM (
  SELECT
    CAST(OrderDate AS TEXT),
    OrderID
  FROM Orders
) AS "t4"
INNER JOIN "Order Details" AS "t5"
  ON "t4"."OrderID" = "t5"."OrderID"
INNER JOIN "Products" AS "t6"
  ON "t5"."ProductID" = "t6"."ProductID"
INNER JOIN "Categories" AS "t7"
  ON "t6"."CategoryID" = "t7"."CategoryID"
```

In [52]:
p_o_details_agg = p_o_details.group_by(["CategoryName", "ProductName"]).aggregate(
    num_orders=p_o_details["OrderID"].count(), 
    total=(p_o_details["Quantity"]*p_o_details["UnitPrice"]*(1-p_o_details["Discount"])).sum()) # cantidad * precio unitario * descuento 
p_o_details_agg = p_o_details_agg.order_by(p_o_details_agg["num_orders"].desc(), p_o_details_agg["total"].desc())
p_o_details_agg # orderna el dataframe por numero de ordenes de forma descendente y el total de forma descendente 
                 # devuelve la consulta de tipo SQL 

r0 := SQLQueryResult
  query:
    SELECT CAST(OrderDate as Text), OrderID FROM Orders
  schema:
    CAST(OrderDate as Text) string
    OrderID                 int64

r1 := DatabaseTable: Order Details
  OrderID   !int64
  ProductID !int64
  UnitPrice !decimal
  Quantity  !int64
  Discount  !float64

r2 := DatabaseTable: Products
  ProductID       !int64
  ProductName     !string
  SupplierID      int64
  CategoryID      int64
  QuantityPerUnit string
  UnitPrice       decimal
  UnitsInStock    int64
  UnitsOnOrder    int64
  ReorderLevel    int64
  Discontinued    !string

r3 := DatabaseTable: Categories
  CategoryID   int64
  CategoryName string
  Description  string
  Picture      binary

r4 := JoinChain[r0]
  JoinLink[inner, r1]
    r0.OrderID == r1.OrderID
  JoinLink[inner, r2]
    r1.ProductID == r2.ProductID
  JoinLink[inner, r3]
    r2.CategoryID == r3.CategoryID
  values:
    CAST(OrderDate as Text): r0['CAST(OrderDate as Text)']
    OrderID:                 r0.OrderID
    ProductID:               r1.ProductID
    UnitPrice:               r1.UnitPrice
    Quantity:                r1.Quantity
    Discount:                r1.Discount
    ProductName:             r2.ProductName
    SupplierID:              r2.SupplierID
    CategoryID:              r2.CategoryID
    QuantityPerUnit:         r2.QuantityPerUnit
    UnitPrice_right:         r2.UnitPrice
    UnitsInStock:            r2.UnitsInStock
    UnitsOnOrder:            r2.UnitsOnOrder
    ReorderLevel:            r2.ReorderLevel
    Discontinued:            r2.Discontinued
    CategoryName:            r3.CategoryName
    Description:             r3.Description
    Picture:                 r3.Picture

r5 := Aggregate[r4]
  groups:
    CategoryName: r4.CategoryName
    ProductName:  r4.ProductName
  metrics:
    num_orders: Count(r4.OrderID)
    total:      Sum(r4.Quantity * r4.UnitPrice * 1 - r4.Discount)

Sort[r5]
  desc r5.num_orders
  desc r5.total

In [53]:
ibis.to_sql(p_o_details_agg) # DEVUELVE UNA CONSULTA DEL TIPO SQL 

```sql
SELECT
  *
FROM (
  SELECT
    "t8"."CategoryName",
    "t8"."ProductName",
    COUNT("t8"."OrderID") AS "num_orders",
    SUM((
      "t8"."Quantity" * "t8"."UnitPrice"
    ) * (
      1 - "t8"."Discount"
    )) AS "total"
  FROM (
    SELECT
      "t4"."CAST(OrderDate as Text)",
      "t4"."OrderID",
      "t5"."ProductID",
      "t5"."UnitPrice",
      "t5"."Quantity",
      "t5"."Discount",
      "t6"."ProductName",
      "t6"."SupplierID",
      "t6"."CategoryID",
      "t6"."QuantityPerUnit",
      "t6"."UnitPrice" AS "UnitPrice_right",
      "t6"."UnitsInStock",
      "t6"."UnitsOnOrder",
      "t6"."ReorderLevel",
      "t6"."Discontinued",
      "t7"."CategoryName",
      "t7"."Description",
      "t7"."Picture"
    FROM (
      SELECT
        CAST(OrderDate AS TEXT),
        OrderID
      FROM Orders
    ) AS "t4"
    INNER JOIN "Order Details" AS "t5"
      ON "t4"."OrderID" = "t5"."OrderID"
    INNER JOIN "Products" AS "t6"
      ON "t5"."ProductID" = "t6"."ProductID"
    INNER JOIN "Categories" AS "t7"
      ON "t6"."CategoryID" = "t7"."CategoryID"
  ) AS "t8"
  GROUP BY
    1,
    2
) AS "t9"
ORDER BY
  "t9"."num_orders" DESC,
  "t9"."total" DESC
```

In [54]:
p_o_details_agg.to_pandas() # AL PONER '.to_pandas()', DEVUELVE EL DATAFRAME QUE ES DEL TIPO 'pandas'  

,CategoryName,ProductName,num_orders,total
0,Condiments,Louisiana Hot Spiced Okra,8040,3504941
1,Confections,Teatime Chocolate Biscuits,8024,1889691.4200000037
2,Beverages,Outback Lager,8020,3064462.65
3,Confections,Sir Rodney's Marmalade,7999,16653807.36
4,Confections,Gumbär Gummibärchen,7999,6391018.984499998
...,...,...,...,...
72,Produce,Tofu,7834,4644738.99
73,Confections,Scottish Longbreads,7818,2528089
74,Meat/Poultry,Perth Pasties,7817,6561681.369999989
75,Dairy Products,Mascarpone Fabioli,7817,6381012.16


In [55]:
p_o_details_agg = p_o_details_agg.order_by(p_o_details_agg["num_orders"].desc(), p_o_details_agg["total"].desc())

In [ ]:
# se utiliza en Ibis 'ibis.desc("category_total")' para ordenar los resultados en orden descendente y considerando el campo 'category_total'. 

In [62]:
consulta= p_o_details_agg.group_by("CategoryName").aggregate(category_total=p_o_details_agg["total"].sum()).order_by(ibis.desc("category_total")) 
consulta.to_pandas() 

,CategoryName,category_total
0,Beverages,92163184.17999999
1,Confections,66337803.06499998
2,Meat/Poultry,64881147.969499916
3,Dairy Products,58018116.78500001
4,Condiments,55795126.78499997
5,Seafood,49921604.16749999
6,Produce,32701119.88000001
7,Grains/Cereals,28568530.3375


In [63]:
ibis.to_sql(consulta) 

```sql
SELECT
  *
FROM (
  SELECT
    "t10"."CategoryName",
    SUM("t10"."total") AS "category_total"
  FROM (
    SELECT
      *
    FROM (
      SELECT
        "t8"."CategoryName",
        "t8"."ProductName",
        COUNT("t8"."OrderID") AS "num_orders",
        SUM((
          "t8"."Quantity" * "t8"."UnitPrice"
        ) * (
          1 - "t8"."Discount"
        )) AS "total"
      FROM (
        SELECT
          "t4"."CAST(OrderDate as Text)",
          "t4"."OrderID",
          "t5"."ProductID",
          "t5"."UnitPrice",
          "t5"."Quantity",
          "t5"."Discount",
          "t6"."ProductName",
          "t6"."SupplierID",
          "t6"."CategoryID",
          "t6"."QuantityPerUnit",
          "t6"."UnitPrice" AS "UnitPrice_right",
          "t6"."UnitsInStock",
          "t6"."UnitsOnOrder",
          "t6"."ReorderLevel",
          "t6"."Discontinued",
          "t7"."CategoryName",
          "t7"."Description",
          "t7"."Picture"
        FROM (
          SELECT
            CAST(OrderDate AS TEXT),
            OrderID
          FROM Orders
        ) AS "t4"
        INNER JOIN "Order Details" AS "t5"
          ON "t4"."OrderID" = "t5"."OrderID"
        INNER JOIN "Products" AS "t6"
          ON "t5"."ProductID" = "t6"."ProductID"
        INNER JOIN "Categories" AS "t7"
          ON "t6"."CategoryID" = "t7"."CategoryID"
      ) AS "t8"
      GROUP BY
        1,
        2
    ) AS "t9"
    ORDER BY
      "t9"."num_orders" DESC,
      "t9"."total" DESC
  ) AS "t10"
  GROUP BY
    1
) AS "t11"
ORDER BY
  "t11"."category_total" DESC
```

In [64]:
precio_por_categoria = consulta.to_pandas() 

fig = px.bar(precio_por_categoria, x='CategoryName', y='category_total') 
fig.show() 

In [59]:
precio_producto = p_o_details_agg.to_pandas()

In [60]:
# precio_producto = p_o_details_agg.to_pandas()

fig = px.bar(precio_producto, x='ProductName', y='total', color='CategoryName') 
fig.show() # EL GRAFICO MUESTRA CUANDO SE GASTA POR PRODUCTO Y LO COLOREA SEGUN CUAL SEA SU CATEGORIA 

### **3. Análisis de Rentabilidad de Clientes**
Analiza la rentabilidad de los clientes de la empresa. Examina los ingresos generados por cada cliente, así como los costos asociados con el servicio o producto proporcionado. 

Pista: Utiliza gráficos de dispersión y tablas dinámicas para identificar los clientes más rentables y aquellos que requieren mayor atención.

In [71]:
# 'Summary of Sales by Quarter' es una vista que ya venía hecha con los datos, revizar programa 'Dbeaver'  
summary_per_quarter = con.sql("SELECT CAST(ShippedDate as Text) as ShippedDate, OrderID, Subtotal FROM [Summary of Sales by Quarter]")

# Las siguientes líneas de código
orders = con.sql("SELECT CustomerID, OrderID FROM Orders")
customers = con.table("Customers")

# Unir los datos
joined_data = summary_per_quarter.join(orders, "OrderID").join(customers, "CustomerID")

# Mostrar el esquema de los datos unidos
print(joined_data.schema()) 

# SELECT CAST convierte el formato de un campo en otro formato 

# Consulta summary_per_quarter: Selecciona ShippedDate, OrderID, y Subtotal de la tabla o vista [Summary of Sales by Quarter], que debe ser una vista o tabla en tu base de datos.

# Consulta orders: Selecciona CustomerID y OrderID de la tabla Orders.

# Tabla customers: Se conecta a la tabla Customers.

# Unir los datos: Une los datos de summary_per_quarter y orders en la columna OrderID, luego une el resultado con customers en la columna CustomerID.

# Mostrar el esquema: Imprime el esquema de los datos unidos para verificar la estructura resultante.

ibis.Schema {
  ShippedDate   string
  OrderID       int64
  Subtotal      float64
  CustomerID    string
  CompanyName   string
  ContactName   string
  ContactTitle  string
  Address       string
  City          string
  Region        string
  PostalCode    string
  Country       string
  Phone         string
  Fax           string
}


In [74]:
per_customer = joined_data.aggregate(
        by="CompanyName",
        total=joined_data["Subtotal"].sum(), 
        dev=joined_data["Subtotal"].std()
    ).order_by(ibis.desc("total")).to_pandas() 

per_customer

,CompanyName,total,dev
0,IT,9.745371e+06,17911.792434
1,B's Beverages,6.154115e+06,17822.000552
2,Hungry Coyote Import Store,5.698024e+06,17304.451978
3,Rancho grande,5.559034e+06,17799.147451
4,Gourmet Lanchonetes,5.552310e+06,17403.553505
...,...,...,...
87,Reggiani Caseifici,4.222835e+06,18400.755160
88,Lehmanns Marktstand,4.182514e+06,18467.964511
89,Furia Bacalhau e Frutos do Mar,4.098648e+06,17470.622273
90,Océano Atlántico Ltda.,4.059079e+06,17213.546773


In [75]:
fig = px.bar(per_customer, x='CompanyName', y='total', error_y='dev') 
fig.show() 

In [81]:
# Definir joined_data
joined_data = summary_per_quarter.join(orders, "OrderID").join(customers, "CustomerID")

# Definir IT_condition. Define IT_condition para especificar el filtro que necesitas aplicar.
IT_condition = joined_data['Region'] == 'IT'  # Ajusta esta condición según tu necesidad

# Filtrado y conversión a Pandas Dataframe 
df_IT_data = joined_data[IT_condition].to_pandas() 

# Conversión de la columna 'ShippedDate' a tipo datetime 
df_IT_data["ShippedDate"] = pd.to_datetime(df_IT_data["ShippedDate"])

# Extración y configuración del índice 
extract = df_IT_data[["ShippedDate", "Subtotal"]].set_index("ShippedDate")

extract_subsampled = extract.resample("BY").mean("Subtotal") 

In [83]:
fig = px.area(extract_subsampled, x="extract_subsampled.index", y="Subtotal") 
fig.show()  

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['Subtotal'] but received: extract_subsampled.index

### **4. Análisis de Inventarios y Proveedores**
Analiza los niveles de inventario y la gestión de proveedores de la empresa. Examina el inventario disponible por categoría de productos, así como el rendimiento de los proveedores en términos de puntualidad de entrega y calidad de los productos. 

Pista: Utiliza gráficos de líneas y tablas dinámicas para mostrar tendencias y comparar el desempeño de los proveedores.

### **5. Análisis de Comportamiento de Empleados**
Analiza el comportamiento de los empleados de la empresa. Examina las ventas realizadas por cada empleado, así como su eficiencia en el manejo de pedidos y su puntualidad en la entrega de productos. 

Pista: Utiliza gráficos de barra y de dispersión para comparar el desempeño de los empleados y identificar áreas de mejora. 

In [109]:
# ESTE CÓDIGO ESTÁ HECHO UTILIZANDO LAS VISTAS QUE SE ENCUENTRAN EN EL PROGRAMA 'DBEAVER' 

empleados_rentables = con.sql("SELECT Salesperson, ExtendedPrice FROM Invoices") 
empleados_rentables.head(5).to_pandas() 

,Salesperson,ExtendedPrice
0,Steven,168.0
1,Steven,98.0
2,Steven,174.0
3,Michael,167.4
4,Michael,1696.0


In [110]:
# ESTO ESTÁ HECHO CON 'IBIS' SIN ESTAR CONECTADO Y EL RESULTADO SE ENVIA A 'PANDAS' PARA SER MOSTRADO 

empleados_rentables = ventas.aggregate(by="Salesperson", rendimiento=ventas["ExtendedPrice"].sum()).order_by(ibis.desc("rendimiento")).to_pandas()
empleados_rentables

,Salesperson,rendimiento
0,Margaret,5.148840e+07
1,Steven,5.138646e+07
2,Janet,5.044557e+07
3,Nancy,4.965942e+07
4,Robert,4.965190e+07
5,Laura,4.928114e+07
6,Michael,4.913997e+07
7,Anne,4.901968e+07
8,Andrew,4.831410e+07


In [111]:
# EL PROGRAMA ESTÁ HECHO CON IBIS, PERO EL RESULTADO NO SE ENVIA A 'PANDAS' POR LO CUAL EL CÓDIGO MUESTRA EL TIPO DE DATO QUE
#  TIENE PARA UNO DE SUS ELEMENTOS 

empleados_rentables = ventas.aggregate(by="Salesperson", rendimiento=ventas["ExtendedPrice"].sum()).order_by(ibis.desc("rendimiento"))
empleados_rentables

r0 := SQLQueryResult
  query:
    SELECT Salesperson, ExtendedPrice FROM Invoices
  schema:
    Salesperson   string
    ExtendedPrice float64

r1 := Aggregate[r0]
  groups:
    Salesperson: r0.Salesperson
  metrics:
    rendimiento: Sum(r0.ExtendedPrice)

Sort[r1]
  desc r1.rendimiento

In [112]:
# LA VARIABLE DE RETORNO DEL CÓDIGO ES INGRESADA COMO PARAMETRO DE ENTRADA A LA FUNCION 'ibis.to_sql()', LA CUAL RETORNA LA 
# CONSULTA SQL QUE PERMITE REALIZAR ESTE MISMO CÓDIGO 

ibis.to_sql(empleados_rentables) 

```sql
SELECT
  *
FROM (
  SELECT
    "t0"."Salesperson",
    SUM("t0"."ExtendedPrice") AS "rendimiento"
  FROM (
    SELECT
      Salesperson,
      ExtendedPrice
    FROM Invoices
  ) AS "t0"
  GROUP BY
    1
) AS "t1"
ORDER BY
  "t1"."rendimiento" DESC
```

In [114]:
empleados_rentables.to_pandas()

,Salesperson,rendimiento
0,Margaret,5.148840e+07
1,Steven,5.138646e+07
2,Janet,5.044557e+07
3,Nancy,4.965942e+07
4,Robert,4.965190e+07
5,Laura,4.928114e+07
6,Michael,4.913997e+07
7,Anne,4.901968e+07
8,Andrew,4.831410e+07
